In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# BÀI TOÁN ĐẶT RA

    Tìm và lọc ra các câu hỏi được xem là không phù hợp trên Quora
    - Input: Các câu hỏi kèm với mã ID ngẫu nhiên
    - Output: 0 (No) và 1 (Yes)
    
    

# THỰC HIỆN BÀI TOÁN
1. Xử lý dữ liệu
- Loại bỏ số, dấu câu và stopword
- Clean lại câu, sửa lại nghĩa 
2. Training dữ liệu
- Sử dụng mô hình Logistic Regression 

# **1, Xử lý dữ liệu:** 
Ở bước này công việc là xử lý từ ngữ của các câu hỏi như là bỏ dấu câu, bỏ số,... sau đó sửa lại câu từ để câu không bị quá khó hiểu

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
import pandas as pd 
import seaborn as sns

**Tổng quan về dữ liệu**
- Bộ dữ liệu gồm 1,3 triệu dòng và 3 cột.
- Các cột bao gồm: qid - unique question identifier (ID riêng của từng câu hỏi), question_text - nội dung của câu hỏi, và target - nếu câu hỏi hợp chuẩn sẽ được đánh 0 và không đạt chuẩn sẽ được đánh 1.
- Bộ dữ liệu có độ chênh lệch giữa câu hợp chuẩn và không hợp chuẩn rất là lớn.

In [ ]:
# Sử dụng df của pandas để xử lý giữ liệu cột và hàng
df_train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
df_test = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
df_sample_sub = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')


In [ ]:
#in ra 20 dòng đầu của  bộ dữ liệu sample_sub
df_sample_sub.head(20) 

In [ ]:
#in ra 20 dòng đầu của bộ dữ liệu train
df_train.head(20) 

In [ ]:
#in ra 20 dòng đầu của bộ dữ liệu test
df_test.head(20) 

Dữ liệu vào là ngôn ngữ tự nhiên. Nếu để nguyên, có vẻ vô cùng khó train và tốn tài nguyên. Yêu cầu đặt ra là cần xử lý các thành phần của câu như dấu câu, số, stopword, và đưa các từ được chia về dạng cơ bản.

In [ ]:
# Kiểm tra xem trong bộ câu hỏi có trường nào trống không của dataframe "df_test"
df_train.info()

In [ ]:
# Kiểm tra số lượng những câu phù hợp với giá trị là 0 và không phù hợp với giá trị là 1
df_train.target.value_counts()

In [ ]:
# Vẽ đồ thị liệt kê số lượng hai loại câu
import matplotlib.ticker as ticker

ncount = df_train.shape[0]

plt.figure(figsize=(7, 5))

ax = sns.countplot(data=df_train, x='target')
plt.title('Portion of Questions')
plt.xlabel('Number of Axles')

# Tạo 2 trục
ax2=ax.twinx()

# Trục ghi số lượng ở bên phải, phần trăm ở bên trái
ax2.yaxis.tick_left()
ax.yaxis.tick_right()


ax.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')

ax2.set_ylabel('Frequency [%]')

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
            ha='center', va='bottom') # set the alignment of the text

# Sử dụng LinearLocator để đảm bảo khoảng cách giữa các mức chính xác cho trục số lượng
ax.yaxis.set_major_locator(ticker.LinearLocator(11))

# Đặt giới hạn phần trăm từ 0 đến 100
ax2.set_ylim(0,100)
ax.set_ylim(0,ncount)

# Sử dụng MultipleLocator để các mức của trục phần trăm cách nhau 10 đơn vị
ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))

ax2.grid(None)

**Xử lý dữ liệu**
Khi xử lý một câu tiếng Anh, vấn đề đầu tiên cần làm và chia tách các từ thành các thành phần riêng biệt. Ngoài ra còn hai vấn đề nữa: một là trong một từ tiếng anh có thể có nhiều dạng chia động từ khác nhau như "drive", "drives, "driving"; và hai là stopword, là những từ như : “a, a, a, of, or, many vv …”, các từ này không hề có liên quan gì đến nội dung và ý nghĩa của câu. Stopword có thể gây ra rất nhiều noise đến việc xử lý từ.
Nên cần một platform xử lý ngôn ngữ tự nhiên đủ mạnh và có đủ các chức năng. Natural Language Toolkit (NLTK) được lựa chọn do có sẵn nhiều thư viện xử lý từ ngữ cho bài toán phân loại. Trong đó tập trung vào 3 phần chính: Word Tokenize, Stopwords và Lemmatizer để xử lý câu hỏi.
* Word Tokenize (Phân đoạn từ): được xử dụng để chia tách các từ thành các thành phần riêng biệt nhằm để có thể nhận diện chính xác từng từ hơn. VD như câu "Chess is one of the oldest known board games." sẽ được xử lý thành ['Chess', 'is', 'one', 'of', 'the', 'oldest', 'known', 'board', 'games', '.']  
* Stopwords: như đã nói ở trên, stopword thường sẽ không liên quan đến nghĩa của câu, để lại sẽ gây ra rất nhiều noise dẫn tới kết quả có sai số lớn. Nên việc cần làm là loại bỏ nó, và NLTK đã có sẵn một list các stopword, có thể lấy ra từ package *nltk.corpus*.
* Lemmatizer: được xử dụng để đưa các từ được chia ra các dạng về dang cơ bản nhất của nó như là "seen", "saw", "seeing" sẽ được chuyển về "see". Việc này cũng sẽ làm giảm noise và giảm tài nguyên, thời gian cần được sử dụng để sử lý. NLTK có sẵn công cụ WordNetLemmatizer cho việc này, có thể lấy ra từ package *nltk.stem*.

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk_stopwords = stopwords.words('english')

wordnet_lemmatizer = WordNetLemmatizer()

# Xây dựng Lemmatizer để đưa các từ về dạng cơ bản
def lemSentence(sentence):
    token_words = word_tokenize(sentence) #tách câu thành từng từ
    lem_sentence = []
    
    for word in token_words:
        lem_sentence.append(wordnet_lemmatizer.lemmatize(word, pos="v")) # Sử dụng "append" để ghi đè các từ được đưa về dạng cơ bản
        lem_sentence.append(" ") # Thêm khoảng trống giữa mỗi từ
        
    return "".join(lem_sentence) 

# Làm "sạch" câu bằng loại bỏ dấu câu, số và stopword; đồng thời đưa các từ về dạng cơ bản
def clean(message, lem=True):
    # Bỏ dấu câu
    message = message.translate(str.maketrans('', '', string.punctuation))
    
    # Bỏ số
    message = message.translate(str.maketrans('', '', string.digits))
    
    # Bỏ stop word
    message = [word for word in word_tokenize(message) if not word.lower() in nltk_stopwords]
    message = ' '.join(message)
    
    # Lemmatazation 
    if lem:
        message = lemSentence(message)
    
    return message

In [ ]:
# Làm sạch câu hỏi với hàm lambda: với mỗi x hãy thực hiện "clean"
df_train['question_text_cleaned'] = df_train.question_text.apply(lambda x: clean(x, True))

**Các câu hỏi sau khi đã được xử lý**


In [ ]:
df_train.head(20)


# **2, Training dữ liệu**
- Với yêu cầu đặt ra, Logistic Regression được lựa chọn vì đây là 1 thuật toán phân loại được dùng để gán các đối tượng cho 1 tập hợp giá trị rời rạc (như 0, 1, 2, ...), và hay được ứng dụng để lọc xem email có phải là spam hay không, có tính tương đồng với bài toán lọc câu hỏi toxic. Nhưng điểm hạn chế của thuật toán này là nó nhạy cảm với noise, điều này đã được xử lý khi ta đưa từ về dạng cơ bản, bỏ stopword, dấu câu và số ở bước trên.
- F1 Score cũng được sử dụng do đâ bộ dữ liệu có qua nhiều mẫu negative thì Accuracy sẽ không còn chính xác nữa.

**----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

* Mặc dù đã xử lý dữ liệu, để sử dụng dữ liệu văn bản cho mô hình dự đoán, văn bản phải được phân tích thành dạng vector. Scikit-learning CountVectorizer được sử dụng để chuyển đổi một bộ các tài liệu văn bản thành một vectơ. Nó cũng cho phép xử lý trước dữ liệu văn bản trước khi tạo biểu diễn vector. Ngoài ra nó cũng cung cấp một cách đơn giản để mã hóa bộ các tài liệu văn bản và xây dựng một kho từ vựng của các từ có sẵn. Vectơ được mã hóa sẽ trả lại độ dài của tự vựng và số lần xuất hiện mỗi từ trong văn bản.
* Pipeline được sử dụng để xâu chuỗi nhiều công cụ đo thành một. Điều này rất hữu ích vì thường có một chuỗi các bước cố định trong quá trình xử lý dữ liệu, đặc biệt là phân loại.

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Tạo hàm chuyển đổi với CountVectorizer
count_vectorizer = CountVectorizer()
model = LogisticRegression(C=1, random_state=0)

# Tạo Pipline với 2 bước
vectorize_model_pipeline = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('model', model)
])

In [ ]:
# Train tập dữ liệu train và test với test_size = 0.3 
X_train, X_test, y_train, y_test = train_test_split(df_train['question_text_cleaned'], df_train['target'], test_size=0.3)
vectorize_model_pipeline.fit(X_train, y_train)

In [ ]:
predictions = vectorize_model_pipeline.predict(X_test)

In [ ]:
# In ra Accuracy và F1 Score
print('Accuracy :', accuracy_score(y_test, predictions))
print('F1 score :', accuracy_score(y_test, predictions))

In [ ]:
#Classificaion report dùng để đo chất lượng dự đoán của thuật toán phân loại. Báo cáo cho thấy độ chính xác của precision, recall và f1-score
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

**Kết quả:** Sử dụng Logistic Regression cho ra kết quả khá cao khi tìm các câu hợp quy chuẩn, nhưng không được cao lắm với những câu không phù hợp.

In [ ]:
# Làm sạch câu 
df_test['question_text_cleaned'] = df_test.question_text.apply(lambda x: clean(x, True))

In [ ]:
df_test['prediction'] = vectorize_model_pipeline.predict(df_test['question_text_cleaned'])

In [ ]:
df_final = df_test[['qid','question_text_cleaned','prediction']]
df_final.set_index('qid', inplace=True)

In [ ]:
# In ra 20 kết quả đầu tiên
df_final.head(20)

In [ ]:
df_final.to_csv('submission.csv')